In [10]:
import os
import numpy as np
import pandas as pd

In [2]:
path="D:\Semester 2\Machine Learning Lab\9"
os.chdir(path)

In [6]:
files=os.listdir()[-2:]

In [233]:
train=np.loadtxt(path+'\\'+files[1],delimiter=',')

In [234]:
# Features in rows and records in rows
train=train.T

In [235]:
# First 60 rows are features and last one row is label for that column
x_train=train[:-1,:]
y_train=train[-1,:]

In [236]:
# Split data into classes
class_0_indices=np.where(y_train==0)[0] # np.where returns a numpy array inside a tuple of length 1, so use index 0 to extract that array
class_1_indices=np.where(y_train==1)[0]

y_train_class_1=y_train[class_1_indices]
y_train_class_0=y_train[class_0_indices]
x_train_class_0=x_train[:,class_0_indices]
x_train_class_1=x_train[:,class_1_indices]

In [237]:
test=np.loadtxt(path+'\\'+files[0],delimiter=',')
test=test.T
x_test=test[:-1,:]
y_test=test[-1,:]

In [238]:
# Divide test data Class wise
class_0_test_indices=np.where(y_test==0)[0] # np.where returns a numpy array inside a tuple of length 1, so use index 0 to extract that array
class_1_test_indices=np.where(y_test==1)[0]

y_test_1=y_test[class_1_test_indices]
y_test_0=y_test[class_0_test_indices]

# Problem 1

In [239]:
# Calculate the mean of the classes
mean_0=x_train_class_0.mean(axis=1)
mean_1=x_train_class_1.mean(axis=1)
# Convert the vectors into column vecotr
mean_0=mean_0.reshape(-1,1)
mean_1=mean_1.reshape(-1,1)

In [240]:
# Covariance of the classes
C0=np.cov(x_train_class_0)
C1=np.cov(x_train_class_1)

In [241]:
C0_inv=np.linalg.inv(C0)
C1_inv=np.linalg.inv(C1)
det_C1=np.linalg.det(C1)
det_C0=np.linalg.det(C0)

In [242]:
# Construct parameters of the discriminant function with Gaussian assumption on instance distribution in classes.
P_X_0=len(y_train_class_0)/len(y_train) # Fraction of points in class 0
P_X_1=len(y_train_class_1)/len(y_train) # Fraction of points in class 1

In [243]:
# g(x) is discriminant function
def g(x):
    if x.shape==(60,):
        x=x.reshape(-1,1)
    value=(np.log(P_X_1/P_X_0)-(1/2)*np.log(det_C1/det_C0)-
           (1/2)*((mean_1.T@C1_inv@mean_1)[0][0]-(mean_0.T@C0_inv@mean_0)[0][0])+
           (x.T@(C1_inv@mean_1-C0_inv@mean_0))[0][0]-(1/2)*(x.T@(C1_inv-C0_inv)@x)[0][0]
          )
    return value
           

In [244]:
def predict(x):
    if g(x)>=0:
        return 1
    else:
        return 0

In [245]:
n=len(y_test) # Number of test samples
predicted=np.zeros(n,dtype=int)
for i in range(n):
    predicted[i]=predict(x_test[:,i])

In [246]:
accuracy=sum(predicted==y_test)/len(y_test)
print("Overall accuracy",accuracy)

Overall accuracy 0.9054656999442275


In [247]:
pred_1=predicted[class_1_test_indices]
pred_0=predicted[class_0_test_indices]         

In [248]:
accuracy_1=sum(pred_1==y_test_1)/len(y_test_1)
print("Class 1 accuracy",accuracy_1)

Class 1 accuracy 0.951007579959327


In [249]:
accuracy_0=sum(pred_0==y_test_0)/len(y_test_0)
print("Class 0 accuracy",accuracy_0)

Class 0 accuracy 0.859412974387736


# Problem 2

In [379]:
w0=np.random.rand(1)
w=np.random.rand(x_train.shape[0],1)

In [380]:
def h(x,w,w0):
    u=((w.T@x)+w0)[0]
    return 1/(1+np.exp(-u))

In [381]:
def gradient(x,y,w,w0):
    grad_w0=(y-h(x,w,w0)).sum()
    grad=((y-h(x,w,w0))*x).sum(axis=1).reshape(60,-1)
    return [grad,grad_w0]

In [382]:
def grad_ascent(x,y,w,w0,eta,iterations):
    for i in range(iterations):
        w=w+eta*gradient(x,y,w,w0)[0]
        w0=w0+eta*gradient(x,y,w,w0)[1]
    return w,w0

In [383]:
params=grad_ascent(x_train,y_train,w,w0,0.0001,5000)

In [384]:
params

(array([[  7.97873245],
        [  3.32616341],
        [  3.19440992],
        [  1.82555978],
        [  3.22431409],
        [ -1.8835814 ],
        [ -3.66808322],
        [  3.16979111],
        [ -3.3214083 ],
        [ -3.83015996],
        [ -2.16752931],
        [ -5.6571392 ],
        [ -3.47467686],
        [ -6.54735604],
        [ -2.33748949],
        [  4.01262413],
        [ -5.16514015],
        [  2.62042855],
        [  3.4719912 ],
        [ -4.24832139],
        [ -1.16288679],
        [ -2.60131186],
        [  2.63987718],
        [ -0.60229273],
        [  0.46677041],
        [ -1.95309816],
        [ -1.13272166],
        [ -1.03789578],
        [  0.59277386],
        [  5.16375132],
        [  3.33045959],
        [  2.02409963],
        [  6.63514607],
        [  0.89823592],
        [  1.11677189],
        [  0.93550478],
        [ -0.36402276],
        [  2.17130763],
        [  1.85571914],
        [ -1.16351733],
        [ 59.50920618],
        [ 20.650

In [385]:
y_test

array([1., 0., 1., ..., 0., 0., 1.])

In [386]:
predicted=np.zeros(len(y_test))
for i in range(len(y_test)):
    x=x_test[:,i].reshape(60,-1)
    if h(x,params[0],params[1])>=0.5:
        predicted[i]=1
    else:
        predicted[i]=0

In [387]:
sum(predicted==y_test)/len(y_test)

0.9283788808328686

In [388]:
pred_1=predicted[class_1_test_indices]
pred_0=predicted[class_0_test_indices] 

In [389]:
accuracy_1=sum(pred_1==y_test_1)/len(y_test_1)
print("Class 1 accuracy",accuracy_1)

Class 1 accuracy 0.959974117212054


In [390]:
accuracy_0=sum(pred_0==y_test_0)/len(y_test_0)
print("Class 0 accuracy",accuracy_0)

Class 0 accuracy 0.8964292391101141
